In [1]:
import pandas as pd
import torch
import re

In [2]:
def filter_arabic_only(text):
    if not isinstance(text, str):
        return ""
    return re.sub(r'[^\u0600-\u06FF\s0-9]', '', text)

In [3]:
datasetsCount = 1
combinedText = []
datasetsFeatures = ["text","story"]
for i in range(datasetsCount):
    df = pd.read_csv(f"../Data/RAW_dataset/ArabicLLM{i + 1}")
    df = df.dropna()
    df = df.drop_duplicates()
    df[datasetsFeatures[i]] = df[datasetsFeatures[i]].astype(str)
    df[datasetsFeatures[i]] = df[datasetsFeatures[i]].apply(filter_arabic_only)
    df[datasetsFeatures[i]] = df[datasetsFeatures[i]]
    
   
    for j in df[datasetsFeatures[i]]:
        combinedText.append(j)

<class 'str'>


In [13]:
combinedText = combinedText[:100000]

In [14]:
len(combinedText)

100000

In [20]:


def clean_arabic_for_tiny_model(text):
    if not text or not isinstance(text, str):
        return ""
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'http\S+|www\.\S+', '', text)
    text = re.sub(r'\S+@\S+', '', text)
    text = re.sub(r'ـ', '', text)
    text = re.sub(r'[\u064B-\u065F]', '', text)
    text = re.sub(r'[أإآ]', 'ا', text)
    numbersMapping = str.maketrans('٠١٢٣٤٥٦٧٨٩', '0123456789')
    text = text.translate(numbersMapping)
    text = text.replace(',', '،').replace('?', '؟')
    text = re.sub(r'(.)\1{2,}', r'\1\1', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def is_valid_line(text, min_words=20):
    if len(text.split()) < min_words:
        return False
    return True

cleanedText = []
for i in combinedText:
    if is_valid_line(i):
        text = clean_arabic_for_tiny_model(i) + " <|endoftext|> "
        cleanedText.append(text)
        

combinedText = cleanedText

In [23]:
count = 0
for i in combinedText:
    if len(i.split()) <= 20:
        count = count + 1
print(count)

142


In [ ]:
combinedText[5]

In [24]:
def countOfWords(lisOFWords:list) -> int:
    counter = 0
    for i in lisOFWords:
        text = i.split()
        counter = len(text) + counter
    return counter

In [25]:
countOfWords(cleanedText)

68777574

In [ ]:
combinedText[0]

In [ ]:
from transformers import AutoTokenizer


tokenizerGPT = AutoTokenizer.from_pretrained("gpt2")

customTokenizer = tokenizerGPT.train_new_from_iterator(combinedText, vocab_size=16000,new_special_tokens=["<QUESTION>","<ANSWER>","<|endoftext|>","[PAD]"],
                                                       initial_alphabet=[])

customTokenizer.add_special_tokens({'pad_token': '[PAD]'})

customTokenizer.save_pretrained("./TOKENIZER")

In [ ]:
import shutil
import os

save_path = "./TOKENIZER/"

if os.path.exists(save_path):
    shutil.rmtree(save_path)
    print("Deleted")

os.makedirs(save_path, exist_ok=True)
customTokenizer.save_pretrained(save_path)
print("Saved")

In [42]:
retrivedTokenizer = AutoTokenizer.from_pretrained("./TOKENIZER/")

In [ ]:
ids = customTokenizer.encode("بوضع".strip())
decoded = customTokenizer.decode(ids)
print(ids)
print("Decoded Text:", decoded)


In [ ]:
print("Decoded Text:", customTokenizer.decode(4373))

In [ ]:

full_tokens = retrivedTokenizer(combinedText, truncation=False, padding=False)["input_ids"]


flattened_ids = [token for sequence in full_tokens for token in sequence]
block_size = 512
total_len = (len(flattened_ids) // block_size) * block_size 
chunks = [flattened_ids[i : i + block_size] for i in range(0, total_len, block_size)]

IDs = torch.tensor(chunks)
masks = torch.ones_like(IDs)

In [15]:
# IDs = outputTokenizer["input_ids"]

# masks = outputTokenizer["attention_mask"]

In [ ]:
torch.save(IDs,"../Data/preproccessedData.pt")
torch.save(masks,"../Data/preproccessedDataMASKS.pt")

tokenCOUNT = len(masks) * 512
tokenCOUNT